In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# base imports
import os
import numpy as np
import pandas as pd

# text cleaning imports
from bs4 import BeautifulSoup
import re
import nltk

# ml imports
import tensorflow as tf
from tensorflow.keras import callbacks, models, layers
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier

# tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Binarizer

# visualizations
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BASE = '/kaggle/input/word2vec-nlp-tutorial'
MAX_WORDS = 25_000

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
train = pd.read_csv(os.path.join(BASE,'labeledTrainData.tsv.zip'), header=0, delimiter="\t", quoting=3)
train.shape

In [ ]:
sns.histplot(data=train, x="sentiment")

In [ ]:
test = pd.read_csv(os.path.join(BASE,'testData.tsv.zip'), header=0, delimiter="\t", quoting=3)
test.shape

In [ ]:
stop_words = stopwords.words("english")
def clean(review):
    clean_html = BeautifulSoup(review).get_text()
    clean_non_letters = re.sub("[^a-zA-Z]", " ", clean_html)
    cleaned_lowercase = clean_non_letters.lower()
    words = cleaned_lowercase.split()
    cleaned_words = [w for w in words if w not in stop_words]
    return " ".join(cleaned_words)

train["cleaned_review"] = train["review"].apply(clean)
train

# Convert to bag of words

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)
binarizer = Binarizer()
X = binarizer.fit_transform(vectorizer.fit_transform(train.cleaned_review)).toarray()
y = train.sentiment
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [ ]:
vocab = vectorizer.get_feature_names()
vocab[:20]

# Gradient Boosted Tree

In [ ]:
model = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(X_train, y_train, 
          early_stopping_rounds=5, 
          eval_set=[(X_val, y_val)], 
          verbose=True)

In [ ]:
pred = model.predict(X_val)
classification_report(y_val, pred)

# Create submission

In [ ]:
test["cleaned_review"] = test["review"].apply(clean)
test

In [ ]:
X_test = binarizer.transform(vectorizer.transform(test.cleaned_review)).toarray()
X_test.shape

In [ ]:
test_values = model.predict(X_test)

In [ ]:
output = pd.DataFrame(data={"id":test.id, "sentiment":test_values})
output.to_csv("bag_of_words_model.csv", index=False, quoting=3)

# Confusion Matrix

In [ ]:
# predictions = model.predict(X_val)
# cf = confusion_matrix(y_val, predictions)
# cf

In [ ]:
# plt.figure(figsize=(12,12))
# sns.heatmap(cf, annot=True, cmap='Blues', robust=True)
# plt.savefig('/kaggle/working/confusion_matrix.png')
# plt.show()